## ⚙️ Package Installation & Version Check

**IMPORTANT:** Cloud-based red teaming requires `azure-ai-projects` version `1.1.0b3` or later.

### Check Your Current Version

Run this cell first to check if you have the correct version installed:

In [1]:
# Check and install/upgrade to the required version
import importlib.util
import subprocess
import sys

def check_and_install():
    needs_install = []
    
    # Check azure-ai-projects version
    try:
        import azure.ai.projects
        if azure.ai.projects.__version__ != "1.1.0b3":
            needs_install.append("azure-ai-projects==1.1.0b3")
    except ImportError:
        needs_install.append("azure-ai-projects==1.1.0b3")
    
    # Check azure-identity
    if importlib.util.find_spec("azure.identity") is None:
        needs_install.append("azure-identity")
    
    # Check python-dotenv
    if importlib.util.find_spec("dotenv") is None:
        needs_install.append("python-dotenv")
    
    if needs_install:
        print(f"📦 Installing/upgrading: {', '.join(needs_install)}")
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q", 
            "--disable-pip-version-check", "--upgrade"
        ] + needs_install)
        print("\n✅ Installation complete! Please restart the kernel and run this notebook from the beginning.")
    else:
        print("✅ All required packages are already installed at the correct versions!")

check_and_install()

📦 Installing/upgrading: azure-ai-projects==1.1.0b3

✅ Installation complete! Please restart the kernel and run this notebook from the beginning.

✅ Installation complete! Please restart the kernel and run this notebook from the beginning.


### Install/Upgrade Required Packages

If the check above shows you need to upgrade, run this cell:

**Note:** This will upgrade your azure-ai-projects package to the beta version required for cloud red teaming.

In [2]:
# Check installed version
try:
    import azure.ai.projects
    print(f"✅ azure-ai-projects version: {azure.ai.projects.__version__}")
    
    # Try to import RedTeam to verify cloud red teaming support
    try:
        from azure.ai.projects.models import RedTeam
        print("✅ Cloud red teaming features available!")
    except ImportError:
        print("❌ Cloud red teaming features NOT available!")
        print("   Your version is too old. You need version 1.1.0b3 or later.")
        print("\n   Please run the installation cell below.")
except ImportError:
    print("❌ azure-ai-projects not installed!")
    print("   Please run the installation cell below.")

✅ azure-ai-projects version: 1.1.0b4
✅ Cloud red teaming features available!


# 🔬 Lab 3: Cloud-Based AI Red Teaming with Azure AI Foundry

**Automated Security Testing of Azure OpenAI Models at Scale**

---

## 🎯 What You'll Learn

In this hands-on lab, you'll discover how to leverage Azure AI Foundry's cloud-based red teaming capabilities to conduct comprehensive security testing of your AI models. Unlike local scanning (Labs 1-2), cloud-based scanning allows you to:

- ✅ **Scale Security Testing**: Run larger combinations of attack strategies and risk categories
- ✅ **Pre-Deployment Validation**: Conduct thorough safety testing before production deployment
- ✅ **Centralized Management**: Submit, monitor, and analyze scans through Azure AI Foundry
- ✅ **Team Collaboration**: Share results and insights with your security team via the portal

### 🔑 Key Differences: Cloud vs. Local Red Teaming

| Aspect | Local (Labs 1-2) | Cloud (This Lab) |
|--------|------------------|------------------|
| **SDK Used** | `azure-ai-evaluation` | `azure-ai-projects` (v1.1.0b3+) |
| **Execution** | Runs on your machine | Runs in Azure AI Foundry |
| **Scale** | Limited by local resources | Scalable cloud infrastructure |
| **Use Case** | Prototyping & development | Pre-deployment & production testing |
| **Results** | Local files + portal | Portal-based analysis |

---

## 📋 Prerequisites

Before starting this lab, ensure you have:

- ✅ **Azure Setup**: Completed environment setup from Lab 0
- ✅ **Environment Variables**: Run `source ../../infra/2-setup-env.sh` to configure
- ✅ **Azure Login**: Authenticated with `az login`
- ✅ **Model Deployment**: Azure OpenAI model deployed in your AI Foundry project
- ✅ **Package Version**: `azure-ai-projects==1.1.0b3` or later (verified in previous cells)

---

## 🗺️ Lab Workflow

This lab guides you through a complete cloud-based red team scanning workflow:

```mermaid
graph LR
    A[Configure Project] --> B[Setup Model Target]
    B --> C[Configure Scan]
    C --> D[Submit to Cloud]
    D --> E[Monitor Progress]
    E --> F[View in Portal]
```

**Steps:**
1. **Environment Setup**: Load credentials and validate configuration
2. **Project Connection**: Connect to Azure AI Foundry project
3. **Model Configuration**: Specify the Azure OpenAI model to test
4. **Scan Configuration**: Define attack strategies and risk categories
5. **Scan Execution**: Submit the scan to Azure cloud infrastructure
6. **Status Monitoring**: Track scan progress programmatically
7. **Results Analysis**: Access detailed findings in Azure AI Foundry Portal

---

## ⏱️ Expected Duration

- **Scan Configuration & Submission**: ~5 minutes
- **Cloud Scan Execution**: ~5-15 minutes (runs in Azure)
- **Results Analysis**: ~10 minutes

**Total Lab Time**: Approximately 20-30 minutes

---

## 📚 Learn More

- **[Run AI Red Teaming Agent in the Cloud](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-ai-red-teaming-cloud)** - Official Azure documentation
- **[AI Red Teaming Agent Concepts](https://learn.microsoft.com/azure/ai-foundry/concepts/ai-red-teaming-agent)** - Understanding attack strategies and risk categories
- **[Azure AI Foundry Projects](https://learn.microsoft.com/azure/ai-foundry/how-to/create-projects)** - Project setup and management

---

Let's begin! 🚀

## 📦 Step 1: Import Required Libraries

In this step, we'll import the necessary Python packages for cloud-based red teaming:

### Package Overview

- **`os`, `sys`, `Path`**: System utilities for file paths and environment management
- **`dotenv`**: Loads environment variables from `.env` file (created by setup script)
- **`azure.identity.DefaultAzureCredential`**: Handles Azure authentication from your `az login` session
- **`azure.ai.projects.AIProjectClient`**: Main client for interacting with Azure AI Foundry projects
- **`azure.ai.projects.models`**: Classes for configuring cloud-based red team scans

### 🔍 Key Difference: Cloud vs. Local Imports

**Cloud-based red teaming** (this lab):
```python
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import RedTeam, AzureOpenAIModelConfiguration
```

**Local red teaming** (Labs 1-2):
```python
from azure.ai.evaluation.red_team import RedTeam
```

These are **different classes** for different execution environments!

---

**Run the cell below to import all libraries:**

In [3]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    RedTeam,
    AzureOpenAIModelConfiguration,
    AttackStrategy,
    RiskCategory,
)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


### 🔍 Troubleshooting Imports

If you encounter import errors, verify your package versions:

```python
import azure.ai.projects
print(f"azure-ai-projects version: {azure.ai.projects.__version__}")
```

**Common issues:**
- **ModuleNotFoundError**: Run `pip install azure-ai-projects==1.1.0b3 azure-identity`
- **ImportError for AttackStrategy/RiskCategory**: Ensure you're importing from `azure.ai.projects.models` (not `azure.ai.evaluation.red_team`)
- The cloud-based red teaming uses different imports than local red teaming (Lab 01)

---

## 🔐 Step 2: Load Environment Configuration

The setup script (`infra/2-setup-env.sh`) created a `.env` file in the repository root with all necessary Azure credentials and endpoints.

### What's in the `.env` File?

- **`AZURE_AI_PROJECT_ENDPOINT`**: Your AI Foundry project endpoint URL
- **`AZURE_OPENAI_ENDPOINT`**: Azure OpenAI service endpoint
- **`AZURE_OPENAI_API_KEY`**: API key for Azure OpenAI authentication
- **`AZURE_AI_DEPLOYMENT_NAME`**: Name of your deployed model (e.g., `gpt-4o`)
- **Other Azure resource identifiers**: Subscription ID, resource group, etc.

### 📂 File Location

The `.env` file is located at: `../../.env` (repository root, two levels up from this notebook)

---

**Run the cell below to load environment variables:**

In [4]:
def load_environment():
    """Load environment variables from .env file in repository root."""
    # Navigate to repository root (two levels up from this notebook)
    env_path = Path.cwd().parent.parent / ".env"
    
    if not env_path.exists():
        print(f"❌ Error: .env file not found at: {env_path}")
        print("\n⚠️  Please run: source ../../infra/2-setup-env.sh")
        return False
    
    load_dotenv(env_path)
    print(f"✅ Loaded environment from: {env_path}")
    return True

# Load the environment
if not load_environment():
    raise Exception("Failed to load environment configuration")

✅ Loaded environment from: /workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/.env


## ✅ Step 3: Validate Required Configuration

Before submitting a cloud scan, we need to verify that all required environment variables are properly configured.

### Why Validate?

- **Prevent Runtime Errors**: Catch missing configuration early
- **Security Verification**: Confirm credentials are loaded correctly
- **Debugging Aid**: Identify configuration issues before scan submission

### Variables We're Checking

| Variable | Description | Example |
|----------|-------------|---------|
| `AZURE_AI_PROJECT_ENDPOINT` | AI Foundry project endpoint | `https://hub-name.services.ai.azure.com/api/projects/project-name` |
| `AZURE_OPENAI_ENDPOINT` | Azure OpenAI service endpoint | `https://your-openai.openai.azure.com` |
| `AZURE_OPENAI_API_KEY` | Authentication key | `abc123...` (displayed as `abc123...***`) |
| `AZURE_AI_DEPLOYMENT_NAME` | Model deployment name | `gpt-4o-mini` |

---

**Run the cell below to validate your configuration:**

In [5]:
def validate_environment():
    """Validate that all required environment variables are set."""
    required_vars = {
        "AZURE_AI_PROJECT_ENDPOINT": "Azure AI Project endpoint URL",
        "AZURE_OPENAI_ENDPOINT": "Azure OpenAI endpoint URL",
        "AZURE_OPENAI_API_KEY": "Azure OpenAI API key",
        "AZURE_AI_DEPLOYMENT_NAME": "Model deployment name",
    }
    
    missing_vars = []
    for var, description in required_vars.items():
        value = os.getenv(var)
        if not value:
            missing_vars.append(f"  ❌ {var}: {description}")
        else:
            # Show partial value for verification (hide sensitive data)
            if "KEY" in var or "SECRET" in var:
                display_value = value[:8] + "..." if len(value) > 8 else "***"
            else:
                display_value = value
            print(f"  ✅ {var}: {display_value}")
    
    if missing_vars:
        print("\n❌ Error: Missing required environment variables:\n")
        print("\n".join(missing_vars))
        print("\n⚠️  Please run: source ../../infra/2-setup-env.sh")
        return False
    
    print("\n✅ All required environment variables are set!")
    return True

# Validate the environment
if not validate_environment():
    raise Exception("Missing required environment variables")

  ✅ AZURE_AI_PROJECT_ENDPOINT: https://aoai-g4o6rvcduotd6.services.ai.azure.com/api/projects/proj-g4o6rvcduotd6
  ✅ AZURE_OPENAI_ENDPOINT: https://aoai-g4o6rvcduotd6.openai.azure.com/
  ✅ AZURE_OPENAI_API_KEY: 85qxLpuB...
  ✅ AZURE_AI_DEPLOYMENT_NAME: gpt-4.1-mini

✅ All required environment variables are set!


## 🔗 Step 4: Extract Azure AI Project Endpoint

The Azure AI Foundry project endpoint is your gateway to all cloud-based AI services, including red teaming.

### Endpoint Structure

Azure AI Foundry project endpoints follow this format:

```
https://<hub-name>.services.ai.azure.com/api/projects/<project-name>
```

Where:
- **`hub-name`**: Your Azure AI Foundry hub resource name
- **`project-name`**: Your specific project within the hub

### Why Do We Need This?

The `AIProjectClient` uses this endpoint to:
- 🔹 Authenticate your requests to Azure AI Foundry
- 🔹 Route API calls to your specific project
- 🔹 Submit and manage red team scans
- 🔹 Access project resources (models, deployments, connections)

---

**Run the cell below to extract and display your project endpoint:**

In [6]:
# Get the AI Project endpoint
project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
project_name = os.getenv("AZURE_AI_PROJECT")

print("📍 Azure AI Project Configuration:")
print(f"   Project Name: {project_name}")
print(f"   Project Endpoint: {project_endpoint}")
print("\n✅ Project connection configured!")

📍 Azure AI Project Configuration:
   Project Name: proj-g4o6rvcduotd6
   Project Endpoint: https://aoai-g4o6rvcduotd6.services.ai.azure.com/api/projects/proj-g4o6rvcduotd6

✅ Project connection configured!


## 🤖 Step 5: Configure Azure OpenAI Model Target

Now we'll specify which Azure OpenAI model deployment the red team scan will test.

### Target Configuration Components

1. **Model Endpoint**: The URL of your Azure OpenAI service
   - Example: `https://your-openai-resource.openai.azure.com`

2. **API Key**: Authentication credential for the Azure OpenAI service
   - Used by the red teaming service to send test prompts to your model

3. **Deployment Name**: The specific model deployment to test
   - Example: `gpt-4o-mini`, `gpt-4`, `gpt-35-turbo`

### 🎯 What Gets Tested?

The red team scan will:
- Send adversarial prompts to **this specific deployment**
- Analyze how the model responds to attack attempts
- Generate a safety report for **this deployment** only

### 💡 Testing Multiple Models

To test different models or deployments:
1. Change the `AZURE_AI_DEPLOYMENT_NAME` in your `.env` file
2. Reload the environment (re-run Step 2)
3. Submit a new scan with a different `display_name`

---

**Run the cell below to configure your model target:**

In [7]:
# Get Azure OpenAI configuration
model_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_deployment_name = os.getenv("AZURE_AI_DEPLOYMENT_NAME")

print("🤖 Azure OpenAI Model Configuration:")
print(f"   Model Deployment: {model_deployment_name}")
print(f"   Model Endpoint: {model_endpoint}")
print(f"   API Key: {model_api_key[:8]}..." if model_api_key else "   API Key: Not set")
print("\n✅ Model target configured!")

🤖 Azure OpenAI Model Configuration:
   Model Deployment: gpt-4.1-mini
   Model Endpoint: https://aoai-g4o6rvcduotd6.openai.azure.com/
   API Key: 85qxLpuB...

✅ Model target configured!


## 🔑 Step 6: Initialize Azure AI Project Client

The `AIProjectClient` is your programmatic interface to Azure AI Foundry. It enables you to:

- 🔹 Submit red team scans to the cloud
- 🔹 Monitor scan status and progress
- 🔹 List all scans in your project
- 🔹 Retrieve scan results and metadata

### Authentication with DefaultAzureCredential

We use `DefaultAzureCredential` which automatically tries multiple authentication methods in order:

1. **Environment variables** (service principal credentials)
2. **Managed identity** (if running in Azure)
3. **Azure CLI** (`az login` session) ← Most common for local development
4. **Visual Studio Code** (VS Code Azure account)
5. **Interactive browser** (fallback if needed)

Since you ran `az login` during setup, authentication will use your Azure CLI session.

### 📚 Learn More

- **[AIProjectClient Reference](https://learn.microsoft.com/python/api/azure-ai-projects/azure.ai.projects.aipro jectclient)** - Full API documentation
- **[DefaultAzureCredential](https://learn.microsoft.com/python/api/azure-identity/azure.identity.defaultazurecredential)** - Authentication methods

---

**Run the cell below to create the project client:**

In [8]:
# Create the AI Project client with Azure credentials
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False)
)

print("✅ Azure AI Project client initialized!")
print("   Using DefaultAzureCredential (from az login)")

✅ Azure AI Project client initialized!
   Using DefaultAzureCredential (from az login)


## ⚙️ Step 7: Configure the Red Team Scan

This is where we define **what** the red team scan will test and **how** it will attack your model.

### Scan Configuration Parameters

#### 1. Attack Strategies (`AttackStrategy.BASE64`)
Specifies the techniques used to probe your model for vulnerabilities.

**BASE64 Attack Strategy:**
- **What it tests**: Can the model be tricked by base64-encoded malicious prompts?
- **How it works**: Encodes harmful requests in base64 to bypass content filters
- **Example**: Instead of "how to make a bomb", it sends `aG93IHRvIG1ha2UgYSBib21i`

**Other Available Attack Strategies:**
- `FLIP`: Character flipping obfuscation
- `LEETSPEAK`: L33t sp34k substitution
- `ROT13`: Character rotation encoding
- `JAILBREAK`: Prompt injection attempts
- Groups: `EASY`, `MODERATE`, `DIFFICULT`

#### 2. Risk Categories (`RiskCategory.VIOLENCE`)
Defines which safety risks to evaluate.

**VIOLENCE Category:**
- Tests if the model generates violent, harmful, or dangerous content
- Evaluates responses for graphic violence, weapons, or physical harm

**Other Available Risk Categories:**
- `HATE_UNFAIRNESS`: Bias, discrimination, hateful content
- `SEXUAL`: Inappropriate or explicit sexual content
- `SELF_HARM`: Self-injury or suicide-related content

#### 3. Display Name (`3-Cloud-Target`)
A unique identifier for this scan in the Azure AI Foundry Portal.

**Naming Convention:**
- Format: `{LabNumber}-{TargetType}-Target`
- Example: `3-Cloud-Target`
- Helps organize and find scans in the portal

#### 4. Target Configuration
Specifies which model deployment to test (configured in Step 5).

### 📊 Expected Scan Duration

With 1 attack strategy × 1 risk category:
- **Cloud execution time**: ~5-10 minutes
- **Number of attacks**: Varies based on strategy complexity

---

**Run the cell below to configure the scan:**

In [9]:
# Create target configuration for the Azure OpenAI model
target_config = AzureOpenAIModelConfiguration(
    model_deployment_name=model_deployment_name
)

# Configure the Red Team Agent
red_team_agent = RedTeam(
    attack_strategies=[AttackStrategy.BASE64],  # Test base64 encoding obfuscation
    risk_categories=[RiskCategory.VIOLENCE],     # Test for violent content
    display_name="3-Cloud-Target",               # Unique scan identifier
    target=target_config,
)

print("⚙️  Red Team Scan Configuration:")
print("   ─────────────────────────────")
print("   Display Name: 3-Cloud-Target")
print("   Target Model:", model_deployment_name)
print("   Attack Strategies: BASE64")
print("   Risk Categories: VIOLENCE")
print("\n✅ Scan configuration ready!")
print("\n💡 This scan will test if the model can be tricked by base64-encoded violent prompts.")

⚙️  Red Team Scan Configuration:
   ─────────────────────────────
   Display Name: 3-Cloud-Target
   Target Model: gpt-4.1-mini
   Attack Strategies: BASE64
   Risk Categories: VIOLENCE

✅ Scan configuration ready!

💡 This scan will test if the model can be tricked by base64-encoded violent prompts.


## 🚀 Step 8: Submit the Red Team Scan to Azure Cloud

Now we'll submit the configured scan to Azure AI Foundry for execution in the cloud.

### What Happens When You Submit?

1. **Scan Submission**: Configuration is sent to Azure AI Foundry
2. **Queue Placement**: Scan enters the processing queue
3. **Attack Generation**: Cloud service generates adversarial prompts based on strategies
4. **Model Testing**: Prompts are sent to your Azure OpenAI deployment
5. **Response Analysis**: Model responses are evaluated for safety risks
6. **Report Generation**: Detailed findings are compiled and stored

### Cloud Execution Benefits

- ✅ **Runs asynchronously**: You don't need to keep your notebook running
- ✅ **Scalable resources**: Leverages Azure's cloud infrastructure
- ✅ **Persistent results**: All data saved to your AI Foundry project
- ✅ **Team access**: Results available to all project collaborators

### Authentication Headers

The `headers` parameter provides your Azure OpenAI credentials to the red teaming service:
- **`model-endpoint`**: Where your model is hosted
- **`api-key`**: Authentication for accessing your model

This allows the cloud service to send test prompts to your model securely.

### 📌 What You'll Get Back

After submission, you'll receive:
- **`scan.name`**: Unique identifier for this scan (auto-generated)
- **`scan.status`**: Current state (`Pending`, `Running`, `Completed`, `Failed`)
- **`scan.display_name`**: The friendly name you provided (`3-Cloud-Target`)

---

**Run the cell below to submit your scan:**

In [10]:
# Create headers with model credentials
# This allows the red team service to call your Azure OpenAI model
headers = {
    "model-endpoint": model_endpoint,
    "api-key": model_api_key
}

print("🚀 Submitting red team scan...")
print("   This may take a moment...\n")

try:
    # Submit the scan
    red_team_response = project_client.red_teams.create(
        red_team=red_team_agent, 
        headers=headers
    )
    
    print("✅ Scan created successfully!")
    print("   ─────────────────────────")
    print(f"   Scan Name: {red_team_response.name}")
    print(f"   Status: {red_team_response.status}")
    
    # Store scan name for later use
    scan_name = red_team_response.name
    
except Exception as e:
    print(f"\n❌ Error submitting scan: {str(e)}")
    print("\n💡 Troubleshooting tips:")
    print("   1. Verify you're logged in: az login")
    print("   2. Check your .env file has correct values")
    print("   3. Ensure the model deployment exists in your project")
    print("   4. Verify you have permissions to create red team scans")
    raise

🚀 Submitting red team scan...
   This may take a moment...

✅ Scan created successfully!
   ─────────────────────────
   Scan Name: badc9180-2e48-416d-96e9-f8470d495fb7
   Status: NotStarted
✅ Scan created successfully!
   ─────────────────────────
   Scan Name: badc9180-2e48-416d-96e9-f8470d495fb7
   Status: NotStarted


## 📊 Step 9: Monitor Scan Progress

Since the scan runs asynchronously in the cloud, we need to check its status periodically.

### Scan Status States

| Status | Description | What to Do |
|--------|-------------|------------|
| **Pending** | Scan queued, waiting to start | Wait and re-check |
| **Running** | Actively testing your model | Wait for completion (5-15 min) |
| **Completed** | Scan finished successfully | View results in portal |
| **Failed** | Error occurred during scan | Check portal for error details |

### Monitoring Tips

- **Re-run this cell** multiple times to check for updates
- **Typical runtime**: 5-15 minutes for a basic scan like this
- **Don't close the notebook**: You can safely close it; scan continues in the cloud
- **Bookmark the scan name**: You'll need it to retrieve results later

### Programmatic Monitoring

The `project_client.red_teams.get(name=scan_name)` method retrieves the latest status from Azure AI Foundry.

---

**Run the cell below to check scan status (re-run as needed):**

In [11]:
# Get the current status of the scan
get_red_team_response = project_client.red_teams.get(name=scan_name)

print("📊 Red Team Scan Status:")
print("   ────────────────────")
print(f"   Scan Name: {scan_name}")
print(f"   Current Status: {get_red_team_response.status}")

if get_red_team_response.status == "Running":
    print("\n   ⏳ Scan is running...")
    print("      This may take several minutes to complete.")
    print("      Re-run this cell to check for updates.")
elif get_red_team_response.status == "Completed":
    print("\n   ✅ Scan completed!")
    print("      Results are ready to view in the portal.")
elif get_red_team_response.status == "Failed":
    print("\n   ❌ Scan failed.")
    print("      Check the portal for error details.")
else:
    print(f"\n   Status: {get_red_team_response.status}")

📊 Red Team Scan Status:
   ────────────────────
   Scan Name: badc9180-2e48-416d-96e9-f8470d495fb7
   Current Status: Starting

   Status: Starting


## 📋 Step 10: List All Red Team Scans in Your Project

This step helps you view and manage all red team scans that have been run in your Azure AI Foundry project.

### Why List All Scans?

- **Historical Tracking**: See all security tests performed over time
- **Comparison**: Compare results across different configurations or models
- **Audit Trail**: Maintain a record of safety testing for compliance
- **Team Visibility**: Share scan history with security and compliance teams

### What You'll See

For each scan:
- **Scan Name**: Auto-generated unique identifier (e.g., `00000000-0000-0000-0000-000000000000`)
- **Status**: Current state (`Pending`, `Running`, `Completed`, `Failed`)
- **Display Name**: Friendly name you provided (e.g., `3-Cloud-Target`)
- **Created Date**: When the scan was submitted (if available)

### Use Cases

- **Find your current scan**: Locate the scan you just submitted
- **Review past scans**: Access results from previous security tests
- **Track scan history**: Monitor your red teaming activities over time

---

**Run the cell below to list all scans:**

In [12]:
print("📋 All Red Team Scans in This Project:")
print("   " + "="*60 + "\n")

scans = list(project_client.red_teams.list())

if not scans:
    print("   No scans found.")
else:
    for idx, scan in enumerate(scans, 1):
        print(f"   {idx}. Scan Name: {scan.name}")
        print(f"      Status: {scan.status}")
        if hasattr(scan, 'created_at') and scan.created_at:
            print(f"      Created: {scan.created_at}")
        if hasattr(scan, 'display_name') and scan.display_name:
            print(f"      Display Name: {scan.display_name}")
        print()

print(f"\n   Total scans: {len(scans)}")

📋 All Red Team Scans in This Project:

   1. Scan Name: f3f4dbf6-1b14-4292-819c-ebd419a11ea7
      Status: Completed
      Display Name: 2D-CustomPrompts-Target

   2. Scan Name: 25940756-d968-4ed6-8cee-5ee88898ce6f
      Status: Completed
      Display Name: 2C-Application-Target

   3. Scan Name: d18266d6-f326-41dc-8159-5880911adac5
      Status: Completed
      Display Name: 2B-Model-Target

   4. Scan Name: ad37fd4f-d10f-4398-aee2-51fd98ce605c
      Status: Completed
      Display Name: 2A-FixedCallback-Target

   5. Scan Name: 2e6add52-5c2b-4566-b087-b12451b97004
      Status: Completed
      Display Name: 1-Agent-Target

   6. Scan Name: c6ff9402-dba4-4fed-a232-e0c30c787e47
      Status: Completed
      Display Name: Agent-Scan

   7. Scan Name: 0171e619-828c-4ddb-b832-05c2846c6a63
      Status: Completed
      Display Name: red-team-cloud-lab03

   8. Scan Name: 4f211bf9-dc21-4a2c-9610-4a8a707ab2a0
      Status: Completed
      Display Name: Lab2d-Custom-Prompt-Scan

   9. Scan 

## 🌐 Step 11: View Results in Azure AI Foundry Portal

The Azure AI Foundry Portal provides a comprehensive, interactive interface for analyzing your red team scan results.

### 🎯 What the Portal Offers

The portal provides four main views for analyzing scan results:

#### 1. 📊 Overview Tab
- **Success Rate**: Percentage of attacks that succeeded
- **Total Attacks**: Number of adversarial prompts tested
- **Risk Summary**: Breakdown by risk category

#### 2. 🎨 Risk Category View
Shows attack success rate by risk type:

![AI Red Teaming Report by Risk Category](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/ai-red-team-report-risk.png)

- **Violence**: Violent or harmful content
- **Hate/Unfairness**: Bias and discrimination
- **Sexual**: Inappropriate sexual content
- **Self-Harm**: Self-injury related content

#### 3. 🔧 Attack Complexity View
Analyzes which types of attacks were most successful:

![AI Red Teaming Report by Attack Complexity](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/ai-red-team-report-attack.png)

- **Simple attacks**: Direct harmful requests
- **Moderate attacks**: Obfuscation techniques (BASE64, etc.)
- **Complex attacks**: Multi-turn jailbreaks

#### 4. 📝 Data Tab
Detailed row-level view of every attack-response pair:

![AI Red Teaming Data View](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/ai-red-team-data.png)

For each attack:
- ✅ **Attack prompt**: What was sent to the model
- ✅ **Model response**: How the model answered
- ✅ **Success/failure**: Whether the attack bypassed safety measures
- ✅ **Attack strategy**: Which technique was used (BASE64, Flip, etc.)
- ✅ **Risk score**: Severity rating
- ✅ **Human feedback**: Thumbs up/down for manual review

#### 5. 💬 Conversation View
Click "View more" to see full multi-turn conversations:

![Conversation History View](https://learn.microsoft.com/en-us/azure/ai-foundry/media/evaluations/red-teaming-agent/ai-red-team-data-conversation.png)

---

### 📚 Learn More

- **[Viewing Red Team Results in Azure AI Foundry](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-ai-red-teaming-cloud#viewing-ai-red-teaming-results-in-azure-ai-foundry-project-preview)** - Detailed portal guide

---

**Run the cell below to get your personalized portal link:**

In [13]:
# Extract project information for portal URL
endpoint_parts = project_endpoint.split('/')
project_name_from_endpoint = endpoint_parts[-1]
hub_domain = endpoint_parts[2].split('.')[0]

# Construct portal URL
portal_url = f"https://ai.azure.com/project/{project_name_from_endpoint}/redteam"

print("🌐 View Results in Azure AI Foundry Portal")
print("   " + "="*70 + "\n")
print(f"   🔗 Portal URL: {portal_url}\n")
print("   📖 Steps to View Results:")
print("   ─────────────────────────")
print("   1. Click the portal URL above (or copy and paste to browser)")
print(f"   2. Navigate to your project: {project_name}")
print("   3. Click on 'Red Team' in the left navigation panel")
print("   4. Find your scan: red-team-cloud-lab03")
print("   5. Click on the scan to view detailed results")
print("\n   📊 What You'll Find:")
print("   ───────────────────")
print("   • Adversarial prompts that were tested")
print("   • Model responses and how they were scored")
print("   • Risk assessment and severity ratings")
print("   • Recommendations for improving model safety")
print("   • Detailed attack logs and analysis")

print("\n✅ Red team scan process complete!")

🌐 View Results in Azure AI Foundry Portal

   🔗 Portal URL: https://ai.azure.com/project/proj-g4o6rvcduotd6/redteam

   📖 Steps to View Results:
   ─────────────────────────
   1. Click the portal URL above (or copy and paste to browser)
   2. Navigate to your project: proj-g4o6rvcduotd6
   3. Click on 'Red Team' in the left navigation panel
   4. Find your scan: red-team-cloud-lab03
   5. Click on the scan to view detailed results

   📊 What You'll Find:
   ───────────────────
   • Adversarial prompts that were tested
   • Model responses and how they were scored
   • Risk assessment and severity ratings
   • Recommendations for improving model safety
   • Detailed attack logs and analysis

✅ Red team scan process complete!


## 🎉 Lab 3 Summary

Congratulations! You've successfully completed **Lab 3: Cloud-Based AI Red Teaming with Azure AI Foundry**.

---

## ✅ What You Accomplished

In this lab, you learned how to:

1. ✅ **Configure Cloud Red Teaming**: Installed `azure-ai-projects` SDK for cloud-based scanning
2. ✅ **Connect to Azure AI Foundry**: Authenticated and connected to your project programmatically
3. ✅ **Target Azure OpenAI Models**: Configured a model deployment as the scan target
4. ✅ **Define Attack Strategies**: Selected BASE64 encoding to test obfuscation vulnerabilities
5. ✅ **Submit Cloud Scans**: Launched red team scans that run asynchronously in Azure
6. ✅ **Monitor Scan Progress**: Checked status and tracked execution
7. ✅ **Access Portal Results**: Learned how to analyze findings in Azure AI Foundry Portal

---

## 🔍 Key Takeaways

### Cloud vs. Local Red Teaming

| Aspect | Local (Labs 1-2) | Cloud (Lab 3) |
|--------|------------------|---------------|
| **Best For** | Prototyping, quick iteration | Pre-deployment, comprehensive testing |
| **Scale** | Limited by local resources | Scalable cloud infrastructure |
| **SDK** | `azure-ai-evaluation` | `azure-ai-projects` |
| **Results** | Local files + portal | Centralized portal analysis |
| **Team Access** | Individual developer | Shared across project team |

### When to Use Cloud Red Teaming

- ☑️ **Before production deployment**: Comprehensive pre-release safety testing
- ☑️ **Regular security audits**: Scheduled safety assessments
- ☑️ **Model updates**: Testing new model versions or deployments
- ☑️ **Large-scale testing**: Multiple attack strategies × risk categories
- ☑️ **Team collaboration**: Sharing results with security/compliance teams

---

## 🎯 Next Steps

### 1. 🔬 Explore Your Scan Results
Visit the Azure AI Foundry Portal (link from Step 11) to:
- Analyze which attacks succeeded
- Review model responses
- Identify safety vulnerabilities
- Get mitigation recommendations

### 2. 🛠️ Try Different Configurations

Experiment with more comprehensive scans:

```python
# More attack strategies
attack_strategies=[
    AttackStrategy.BASE64,
    AttackStrategy.LEETSPEAK,
    AttackStrategy.JAILBREAK
]

# Multiple risk categories
risk_categories=[
    RiskCategory.VIOLENCE,
    RiskCategory.HATE_UNFAIRNESS,
    RiskCategory.SEXUAL,
    RiskCategory.SELF_HARM
]
```

### 3. 📊 Compare Models
Test different model deployments to compare their safety:
- GPT-4 vs GPT-3.5
- Base models vs fine-tuned models
- Different model versions

### 4. 🔄 Integrate into CI/CD
Automate red teaming in your deployment pipeline:
- Run scans before production deployments
- Set safety thresholds for automated approval
- Block deployments that fail safety tests

### 5. 🧪 Continue to Lab 4 (Optional)
Explore advanced red teaming scenarios and additional attack techniques!

---

## 📚 Additional Resources

### Official Documentation
- **[Azure AI Red Teaming Agent Overview](https://learn.microsoft.com/azure/ai-foundry/concepts/ai-red-teaming-agent)** - Comprehensive guide
- **[Cloud Red Teaming How-To](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-ai-red-teaming-cloud)** - Step-by-step instructions
- **[Attack Strategies Reference](https://learn.microsoft.com/azure/ai-foundry/concepts/ai-red-teaming-agent#attack-strategies)** - All available techniques
- **[Risk Categories Reference](https://learn.microsoft.com/azure/ai-foundry/concepts/ai-red-teaming-agent#risk-categories)** - Safety classifications

### Best Practices
- **[Microsoft AI Red Team Insights](https://www.microsoft.com/security/blog/2025/01/13/3-takeaways-from-red-teaming-100-generative-ai-products/)** - Lessons from 100+ product tests
- **[Planning Red Teaming for LLMs](https://learn.microsoft.com/azure/ai-foundry/openai/concepts/red-teaming)** - Strategic guidance

### Tools and Frameworks
- **[PyRIT Framework](https://github.com/Azure/PyRIT)** - Python Risk Identification Tool (underlying technology)
- **[Azure AI Content Safety](https://learn.microsoft.com/azure/ai-services/content-safety/overview)** - Mitigation filters
- **[Safety System Messages](https://learn.microsoft.com/azure/ai-foundry/openai/concepts/safety-system-message-templates)** - Pre-built safety prompts

---

## 💡 Need Help?

- **Documentation Issues?** Check [Azure AI Foundry Docs](https://learn.microsoft.com/azure/ai-foundry/)
- **Technical Questions?** Review the [Troubleshooting Guide](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/run-ai-red-teaming-cloud#troubleshooting)
- **Scan Failed?** Verify permissions and storage account configuration in Step 3

---

**Great work!** You've mastered cloud-based AI red teaming. 🚀